In [1]:
import sys
import json
import os

sys.path.append(os.path.abspath("../src"))
PROJECT_ROOT = os.getenv("PROJECT_ROOT")

In [2]:
from recommender.data_processors.summoner_champion_processor import SummonerMixProcessor
from recommender.data_loaders.summoner_match_loader import SummonerMatchLoader
from recommender.utils.riot_api_helper import RiotApiHelper
from recommender.utils.map_helper import MapHelper

smp = SummonerMixProcessor()
rah = RiotApiHelper()
sml = SummonerMatchLoader()
mh = MapHelper()

In [3]:
puuid = "_2dRYSt03dHrJrRDcYImk2_umcnVSo8i-1WMun-j8YAFYk9CzauMu7WjQ8IJCY_RDWMopccQI_qfcw"
match_ids = rah.get_player_matches(puuid)

In [6]:
await smp.async_load_encoded_ratings()

,puuid,champ_name,rating,user_id,champ_id,summoner_rank,summoner_lane,champ_attack_type,champ_adaptive_type
0,--EEhXPKpjfGXqRUXQeFUIrad-z3jgjyrwQj6Z49G4zWAK...,Aatrox,8.859117,0,0,GRANDMASTER,MIDDLE,MELEE,PHYSICAL_DAMAGE
1,-0tFbkE3vD6takqkFfPNO0xIRQsE-5GeKCJEBRYGwi04RT...,Aatrox,2.290347,1,0,CHALLENGER,MIDDLE,MELEE,PHYSICAL_DAMAGE
2,-7e5FbjuTrjsYyDu8GQG0p9wORit7_GD7BHCxR-k9rNo6c...,Aatrox,0.000000,2,0,DIAMOND,JUNGLE,MELEE,PHYSICAL_DAMAGE
3,-AeAsdpxcfQCzruP8eiXWNoatCvBehRa4jQHi8EwcOS-o2...,Aatrox,6.300402,3,0,DIAMOND,MIDDLE,MELEE,PHYSICAL_DAMAGE
4,-Bb1a-1E8rA-pPj9Ox4CMqD9HFhUc621aLpnJCb-L0uHMH...,Aatrox,5.728852,5,0,MASTER,JUNGLE,MELEE,PHYSICAL_DAMAGE
...,...,...,...,...,...,...,...,...,...
146195,_vuXUF_TQs1RuindiNch6El_TrJX9b609fJGbXH64046v3...,Yorick,1.861314,501,161,GRANDMASTER,TOP,MELEE,PHYSICAL_DAMAGE
146196,_wcrNrVvGYv3syJjVhNZx93SZJOwl3GYn0vK3oOxQb-0Jh...,Yorick,0.000000,502,161,CHALLENGER,SUPPORT,MELEE,PHYSICAL_DAMAGE
146197,_xGyR0kT2240JuSOwzokb0ewFGVR3VeeGNJMBsYk3oU6Di...,Yorick,0.000000,503,161,DIAMOND,SUPPORT,MELEE,PHYSICAL_DAMAGE
146198,_Xuv4oW8ZMrsulBBFw8neDnnzf517VhDK9gKsGzUHG5YnS...,Yorick,3.870494,495,161,DIAMOND,JUNGLE,MELEE,PHYSICAL_DAMAGE


In [ ]:
# Step 1: Build the df with column of lets say mode rank, mode lanes, roles, attack_types
# Step 2: Make train and test loader using TwoTowerChampionDataset
# Step 3: iterate for user_feattures, champ_features, rating in loader: preds = model(user_features, champ_features)